# Домашнее задание №2: линейная регрессия (10 баллов).

Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [1]:
import numpy as np

### Считаю свой вариант

In [2]:
print((len("Kazakbaev") % 4) + 1)

2


## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [3]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
mse_lin = mean_squared_error(y, reg.predict(X))
print(mse_lin)

1.8281308819724393e-25


In [5]:
np.mean((y-reg.predict(X))**2)

1.8281308819724393e-25

Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [6]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

3.834495168221677e-12


array([-7.57619106e-09,  7.19932809e-09,  3.22765741e-08,  1.19970327e-08,
       -5.35534458e-08,  3.79878232e+01, -2.46688805e-08, -8.35478173e-08,
       -2.69081580e-08,  9.55179619e-09,  1.51937975e-08,  3.52771717e-08,
       -7.42943194e-09,  9.47226298e-09,  1.64842549e-08,  3.56980964e-08,
       -2.72165408e-08,  2.58367876e-08, -2.24285662e-08,  5.92176538e+01,
        5.23182668e-09, -1.82645736e-09, -1.78241088e-08,  9.22875184e+01,
        1.06139762e-08, -4.62962145e-08,  3.11046692e-08, -4.51692844e-08,
       -2.19211855e-08, -4.09619883e-08, -2.56861772e-08, -6.79376171e-09,
        7.41519953e+01,  6.19539814e+00,  5.64223685e-08, -2.17424283e-08,
        8.06539928e-09,  4.03653071e-08,  2.17329518e-08,  3.78904802e-08,
       -2.78752696e-08,  2.08864222e-08,  1.62772378e-08, -2.96220735e-08,
       -4.05992826e-08, -2.12686709e-08,  2.81444337e-08, -1.53097581e-08,
       -1.11333865e-08,  5.59267674e-09,  6.50919588e-09,  4.19532297e+00,
        1.49316947e-08,  

***Задание 1 (1 балл).*** Объясните, чем вызвана разница в значениях двух полученных значений метрики?

***Задание 2 (1 балл).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

### Задание 1

Обратимся, к документации. Просто Linear Regression минимизирует квадрат разницы того, что неблюдается и то, что предсказано. В то же время, как SGDRegressor сдвигается в сторону антиградиента с определенным шагом - learning rate, и с наличием penalty. У SGDRegressor есть penalty который по дефолту стоит L2, у Linear regression этого нет. 

Коэффицент $\alpha$ это множитель регулирязатора(penalty), то есть то насколько сильно мы штрафуем за слишком большие веса. Уменьшая/увеличивая $\alpha$ будем получать разные MSE.

Соответсвенно будет и другой минимизируемая функция: $Q = \frac {1} {l} \sum ((w, x_i) - y_i) ^ 2 + \alpha * \sum (w_i) ^2$

И соотвественно разные MSE

### Задание 2

In [7]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

3.824314905332355e-12


Это наиболее близкое, что я подбором смог сделать. Сокращая альфу, мы сокращаем MSE. В любом случае получившиеся значение это очень маленькое число.

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и четвертый варианты) или l2 (второй и третий варианты) регуляризацию. 

### Задание 3

In [8]:
import sys
epsilon = sys.float_info.epsilon
class LinearRegression(object):
    def __init__(self, learning_rate=0.01, l_ratio=0.001, tol=0.001, epochs=1500, weights=np.zeros(X.shape[1]),
        costs = [0]):
        '''
        Для начала необходимо инициализировать параметры
        learning_rate - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.learning_rate = learning_rate
        self.l_ratio = l_ratio
        self.tol = tol
        self.epochs = epochs  
        self.costs = costs
        self.weights = weights
    def fit(self, X, y_true = y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        weights = self.weights
        costs = self.costs
        learning_rate = self.learning_rate
        N = float(len(y_true))
        for i in range(self.epochs):
            y_pred = np.dot(X, weights)
            cost = 1 / N * np.sum((y_true - y_pred)** 2)
            weights_gradient = (-2 / N) * np.dot(X.T, (y_true - y_pred)) 
            weights = weights - learning_rate * weights_gradient
            costs.append(cost)
            if abs(costs[-1] - costs[-2]) < epsilon:
                break
        return weights
   
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        y_pr = np.dot(X, self.fit(X, y))
        return y_pr

In [9]:
my_reg = LinearRegression()
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')

You are amazing! Great work!


### Задание 4

In [10]:
import sys
epsilon = sys.float_info.epsilon
class LinearRegression_l2(object):
    def __init__(self, learning_rate=0.01, l_ratio=0.1, tol=0.001, epochs=10000, weights=np.zeros(X.shape[1]),
        costs = [0]):
        '''
        Для начала необходимо инициализировать параметры
        learning_rate - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.learning_rate = learning_rate
        self.l_ratio = l_ratio
        self.tol = tol
        self.epochs = epochs  
        self.costs = costs
        self.weights = weights
    def fit(self, X, y_true = y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        weights = self.weights
        costs = self.costs
        learning_rate = self.learning_rate
        l_ratio = self.l_ratio
        N = float(len(y_true))
        for i in range(self.epochs):
            y_pred = np.dot(X, weights)
            cost = 1 / N * np.sum((y_true - y_pred)** 2) + l_ratio * np.sum(weights ** 2)
            weights_gradient = (-2 / N) * np.dot(X.T, (y_true - y_pred)) + (l_ratio * 2 * weights) / N
            weights = weights - learning_rate * weights_gradient
            costs.append(cost)
            if abs(costs[-1] - costs[-2]) < epsilon:
                break
        return weights
   
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        y_pr = np.dot(X, self.fit(X, y))
        return y_pr

In [11]:
my_reg = LinearRegression_l2()
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work! even with l2 ratio')

You are amazing! Great work! even with l2 ratio


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки с l1-регуляризацией (from sklearn.linear_model import Lasso, первый и четвертый варианты) или с l2-регуляризацией (from sklearn.linear_model import Ridge, второй и третий варианты) с значением параметра регуляризации 0.1. Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [12]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1).fit(X, y)
print('MSE:', mean_squared_error(y, ridge.predict(X)))
print('Weights:', ridge.coef_)

MSE: 3.7556571658919374e-06
Weights: [-1.88869862e-05 -1.22093245e-05 -2.40969034e-05 -9.57000920e-06
 -2.02034603e-05  3.79874415e+01  4.48751964e-06 -2.57362717e-05
  4.15801608e-06  1.67372682e-05  2.76461837e-06  1.29957328e-05
 -1.95132052e-05  2.44014463e-05  2.12160671e-05 -5.23635899e-06
 -1.84788402e-05 -9.47761762e-07  6.72233803e-06  5.92170225e+01
 -8.72839489e-06 -6.81602373e-06 -5.59098910e-06  9.22865896e+01
  2.34463654e-05 -2.59570690e-05  1.63509586e-06 -1.97562776e-05
  2.36706051e-05 -2.12230950e-05 -3.84037398e-06  5.16850476e-07
  7.41512499e+01  6.19534396e+00 -6.71984550e-06  1.84208151e-06
  1.97558578e-05  3.72840887e-05 -7.06500110e-06  8.34218554e-06
  1.43288430e-05 -4.96375366e-06  4.08028264e-05 -1.49317782e-05
 -4.17698406e-06 -2.02320172e-05 -3.41147058e-06 -1.58049915e-05
 -5.01657156e-06  3.96720540e-05  2.89670270e-06  4.19525858e+00
  3.26091424e-05  1.41747909e-05  1.72842797e-05  3.67558208e-06
 -2.51164161e-07  3.75725496e-05  7.57629330e-06  7.3

In [13]:
my_reg = LinearRegression_l2()
print('MSE:', mean_squared_error(y, my_reg.predict(X)))
print('Weights:', my_reg.fit(X, y))

MSE: 3.755612052048586e-06
Weights: [-1.87872977e-05 -1.22371185e-05 -2.40878115e-05 -9.49741419e-06
 -2.01935811e-05  3.79874415e+01  4.56636036e-06 -2.56372960e-05
  4.13593268e-06  1.68358596e-05  2.75271914e-06  1.29567935e-05
 -1.95273871e-05  2.43789105e-05  2.12466650e-05 -5.19909605e-06
 -1.84203984e-05 -8.28876418e-07  6.65427618e-06  5.92170224e+01
 -8.57304558e-06 -6.80485031e-06 -5.45764453e-06  9.22865896e+01
  2.34957081e-05 -2.58957105e-05  1.57445020e-06 -1.98544269e-05
  2.36495058e-05 -2.13087615e-05 -3.94174972e-06  4.12538609e-07
  7.41512498e+01  6.19534392e+00 -6.73367142e-06  1.82692763e-06
  1.98666494e-05  3.72595637e-05 -7.12106741e-06  8.40951913e-06
  1.43421395e-05 -4.88998276e-06  4.07876129e-05 -1.49774841e-05
 -4.13150565e-06 -2.02633366e-05 -3.37113456e-06 -1.57852384e-05
 -4.93841266e-06  3.97318469e-05  3.03082597e-06  4.19525873e+00
  3.27230381e-05  1.41719367e-05  1.72758086e-05  3.64947440e-06
 -2.42234137e-07  3.74674763e-05  7.39631164e-06  7.31

## Вывод

Получились почти что одинаковые MSE, значит все ок!